In [ ]:
pip install -r requirements.txt

In [ ]:
import skimage
import matplotlib.pyplot as plt
# little heplper code i wrote to show matrix representation
from support import plot

#load image
img = skimage.io.imread('images/banana.png')
plt.imshow(img)

In [ ]:
from skimage import transform
# rotate by 45 degrees
img_rot = transform.rotate(img, 45)
plt.imshow(img_rot)

In [ ]:
# generate 4 images, rotated by 15, 30, 45, 60 degrees

img_rot15 = transform.rotate(img, 15)
img_rot30 = transform.rotate(img, 30)
img_rot45 = transform.rotate(img, 45)
img_rot60 = transform.rotate(img, 60)

# display the images

fig, ax = plt.subplots(1, 4, figsize=(20, 20))
ax[0].imshow(img_rot15)
ax[1].imshow(img_rot30)
ax[2].imshow(img_rot45)
ax[3].imshow(img_rot60)

In [ ]:
from skimage import util, transform, graph, filters, color
import numpy as np

def threshold_image(img, t: float):
    return np.where(img > t, 1., 0)

#load image
img = skimage.io.imread('images/banana.png')[:,:,0:3]
img = util.crop(img, ((150, 150), (20, 20), (0, 0)))
img = color.rgb2gray(img)
img = threshold_image(img, 0.3)
plt.imshow(img)


In [ ]:
from skimage.util import view_as_windows
img = skimage.io.imread('images/banana.png')[:,:,0:3]
# Define the scales for the sliding window
scales = [(img.shape[0] // 3,img.shape[1] // 3)]

# scale image to 1024x1024
img = transform.resize(img, (1024, 1024))
fig, axes = plt.subplots(len(scales), figsize=(5, 5*len(scales)))

# make array if axes is scalar
if not hasattr(axes, '__iter__'):
    axes = [axes]

# iterate over scales
for (scale_x, scale_y), ax in zip(scales, axes):
    ax.imshow(img, cmap='gray')

    step_x = scale_x // 4 * 3
    step_y = scale_y // 4 * 3
    
    for y in range(0, img.shape[0] - scale_y + 1, step_y):
        for x in range(0, img.shape[1] - scale_x + 1, step_x):
            rect = plt.Rectangle((x, y), scale_x, scale_y, edgecolor='blue', facecolor='none')
            ax.add_patch(rect)

plt.show()

In [ ]:
import skimage
from support.bounding_boxes import draw_bounding_boxes
import numpy as np
import matplotlib.pyplot as plt

img = skimage.io.imread('images/Items.jpg')[:,:,1]
print(img.shape)
scales = [(0.8, 8), (0.6, 8), (0.5, 8), (0.4, 8)]

score_threshold = 0.1


plt.imshow(img < 64, cmap='gray')
plt.show()
# calculate score per rectangle
def score(rect, img):
    x, y, w, h = rect
    patch = img[y:y+h, x:x+w]
    return np.mean(patch < 64)

# add rectangle over whole image for initalization
rects_with_object = [(0, 0, img.shape[1], img.shape[0])]
scored_rectangles = dict()

for scale, stride in scales:
    # create patches of image with scale
    height = int(img.shape[0] * scale)
    width = int(img.shape[1] * scale)
    new_rectangles = []
    stride_x = width // stride
    stride_y = height // stride
    for rect in rects_with_object:
        x_o, y_o, w_o, h_o = rect

        # get roi rectangle coordinates for current rectangle
        for x in range(x_o, x_o + w_o - width + 1, stride_x):
            for y in range(y_o, y_o + h_o - height + 1, stride_y):
                # calculate score
                s = score(rect, img)
                if s > score_threshold:
                    new_rectangles.append((x, y, width, height))
                    scored_rectangles[rect] = s
    rects_with_object = new_rectangles
    # plot rectangles on image
    plt.imshow(img, cmap='gray')
    draw_bounding_boxes(new_rectangles)
    
    plt.show()


In [ ]:
from support.bounding_boxes import non_maximum_suppression, draw_bounding_boxes

iou_threshold = 0.3

# do NMS over rectangles


# apply NMS
rects_with_object = non_maximum_suppression(scored_rectangles, overlap_threshold=iou_threshold)

plt.imshow(img, cmap='gray')
draw_bounding_boxes(rects_with_object)
plt.show()

In [ ]:
import skimage
from skimage.morphology import erosion, dilation, disk
import numpy as np
from support.bounding_boxes import draw_bounding_boxes, create_bounding_boxes

img = skimage.io.imread('images/Items.jpg')[:,:, :3]

plt.imshow(img)
plt.show()

# convert to grayscale
img = skimage.color.rgb2gray(img)

plt.imshow(img, cmap='gray')
plt.show()

img = skimage.util.crop(img, ((300, 300), (300, 50)))

plt.imshow(img, cmap='gray')
plt.show()

# scale image
img = skimage.transform.resize(img, (250, 400))

plt.imshow(img, cmap='gray')
plt.show()

mask = img < 0.5

plt.imshow(mask, cmap='gray')
plt.show()

labels = skimage.measure.label(mask)
plt.imshow(labels, cmap='tab20')
plt.show()

rectangles = create_bounding_boxes(labels)


draw_bounding_boxes(rectangles)
plt.imshow(labels, cmap='tab20')
plt.show()

In [ ]:
rectangles

In [ ]:
img = skimage.io.imread('images/Items.jpg')[:,:,1]
img = skimage.util.crop(img, ((400, 1600), (800, 2350)))
for t in [16, 32, 64, 80]:
    filled = skimage.segmentation.flood_fill(img, (img.shape[0] // 2, img.shape[1] // 2), 255, tolerance = t)
    plt.imshow(filled, cmap='gray')
    plt.show()
